# Factorized parameter estimation with precession and higher modes

Run parameter estimation by marginalizing the extrinsic parameters during sampling.

Reference: https://arxiv.org/abs/2404.02435

In [ ]:
# Ensure only one CPU is used:
import os
os.environ["OMP_NUM_THREADS"] = "1"

from pathlib import Path
import pandas as pd

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal
lal.swig_redirect_standard_output_error(False)

from cogwheel import posterior
from cogwheel import sampling
from cogwheel import data
from cogwheel import gw_plotting

%matplotlib widget

In [ ]:
parentdir = Path('example')  # PE runs will be saved here

In [ ]:
eventname = 'GW190412'

In [ ]:
# Create and save `EventData` file
if not data.EventData.get_filename(eventname).exists():
    filenames, detector_names, tgps = data.download_timeseries(eventname)
    event_data = data.EventData.from_timeseries(
        filenames, eventname, detector_names, tgps, t_before=16., fmax=1024.)
    event_data.to_npz()

Note that below we pass `CartesianIntrinsicIASPrior`. The `Intrinsic*` priors will automatically use likelihood marginalization over extrinsic parameters. The `Cartesian*` priors use a spin parametrization that does not use periodic angles (better for samplers such as `nautilus` that don't support periodic coordinates).

In [ ]:
%%time
# Find reference waveform and compute summary data:
post = posterior.Posterior.from_event(
    event=eventname,
    mchirp_guess=data.EVENTS_METADATA['mchirp'][eventname],
    approximant='IMRPhenomXPHM',
    prior_class='CartesianIntrinsicIASPrior')

In [ ]:
# Instantiate sampler, optionally edit parameters
sampler = sampling.Nautilus(post)
sampler.run_kwargs['n_live_points'] = 1000
# sampler.run_kwargs['pool'] = 2  # Optional: parallelize

In [ ]:
%%time
# Run sampler (will take a while)
rundir = sampler.get_rundir(parentdir)
sampler.run(rundir)

In [ ]:
samples = pd.read_feather(rundir/sampling.SAMPLES_FILENAME)

params = [
    'mchirp',
    'lnq',
    'chieff',
    'cumchidiff',
    'd_luminosity',
    'ra',
    'dec',
    'phi_ref',
    'psi',
    't_geocenter',
    'lnl_marginalized',
    'lnl'
]

gw_plotting.CornerPlot(samples, params=params, tail_probability=1e-4).plot(title=eventname)